# Import

In [1]:
import os
import numpy as np
import tensorflow as tf

# Create graph

In [2]:
with tf.gfile.FastGFile(os.path.join(
    '../../../data/inception-2015-12-05/classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')
g = tf.get_default_graph()

# Print tensor names

In [3]:
with tf.Session() as sess:
    # Print tensor names
    tensor_names = [n.name for n in g.as_graph_def().node]
    for t in tensor_names:
        print(t)

DecodeJpeg/contents
DecodeJpeg
Cast
ExpandDims/dim
ExpandDims
ResizeBilinear/size
ResizeBilinear
Sub/y
Sub
Mul/y
Mul
conv/conv2d_params
conv/Conv2D
conv/batchnorm/beta
conv/batchnorm/gamma
conv/batchnorm/moving_mean
conv/batchnorm/moving_variance
conv/batchnorm
conv/CheckNumerics
conv/control_dependency
conv
conv_1/conv2d_params
conv_1/Conv2D
conv_1/batchnorm/beta
conv_1/batchnorm/gamma
conv_1/batchnorm/moving_mean
conv_1/batchnorm/moving_variance
conv_1/batchnorm
conv_1/CheckNumerics
conv_1/control_dependency
conv_1
conv_2/conv2d_params
conv_2/Conv2D
conv_2/batchnorm/beta
conv_2/batchnorm/gamma
conv_2/batchnorm/moving_mean
conv_2/batchnorm/moving_variance
conv_2/batchnorm
conv_2/CheckNumerics
conv_2/control_dependency
conv_2
pool/CheckNumerics
pool/control_dependency
pool
conv_3/conv2d_params
conv_3/Conv2D
conv_3/batchnorm/beta
conv_3/batchnorm/gamma
conv_3/batchnorm/moving_mean
conv_3/batchnorm/moving_variance
conv_3/batchnorm
conv_3/CheckNumerics
conv_3/control_dependency
conv_3
con

# Insert own output layer

In [4]:
pool_3_reshape = g.get_tensor_by_name('pool_3/_reshape:0')
with tf.name_scope('own'):
    our_weights = tf.Variable(tf.random_normal(
        [int(pool_3_reshape.get_shape()[-1]),300], stddev=0.1), name='our_weights')
    our_output = tf.matmul(pool_3_reshape, our_weights, name='our_output')

# Run model

In [5]:
# Read in labels
labels_file = open('../../../data/imagenet_comp_graph_label_strings.txt', 'r')
labels = labels_file.readlines()

with tf.Session() as sess:
    # Predict image
    softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
    image = tf.gfile.FastGFile('../../../data/inception-2015-12-05/boat.jpg', 'rb').read()
    predictions = sess.run(softmax_tensor,{'DecodeJpeg/contents:0': image})
    predictions = np.squeeze(predictions)
    index = np.argmax(predictions)
    print(labels[index])
    
    # Print out output
    own_tensor = sess.graph.get_tensor_by_name('own/our_output:0')
    sess.run(our_weights.initializer)
    print(sess.run(our_output,feed_dict={'DecodeJpeg/contents:0': image}))
    
    # Write to file in order to visualize graph in tensorflow
    sum_writer = tf.summary.FileWriter('./sums', sess.graph)

trimaran

[[  1.81164384e+00  -1.06722426e+00   1.99532855e+00   2.34999418e-01
    1.06127751e+00   1.19117904e+00  -7.72125542e-01  -7.22711802e-01
    5.99767447e-01   7.36469448e-01  -1.31111276e+00  -1.23710454e+00
   -9.86552835e-01   1.17722988e+00  -1.43094406e-01   1.36131179e+00
   -9.01542485e-01  -2.24545479e+00   6.91798091e-01  -1.19202673e+00
    2.74224114e+00   9.65514839e-01  -1.98164785e+00   5.99468529e-01
    2.06221938e+00   5.11921823e-01   9.84064758e-01   4.78344023e-01
    8.69753182e-01   1.63331795e+00  -9.10966694e-01   1.55497622e+00
    1.12386656e+00   2.74027205e+00  -3.24259132e-01   1.69084036e+00
   -7.18190789e-01   1.60876238e+00   9.99185264e-01  -8.11962247e-01
    1.18144906e+00  -8.97268474e-01  -7.77535319e-01  -1.38973385e-01
   -4.91892338e-01   2.85823822e-01  -2.42945719e+00  -1.52662098e+00
   -2.10447168e+00   5.54251194e-01  -3.84976554e+00  -2.92857361e+00
    1.03314435e+00  -4.02091831e-01  -5.80611765e-01   2.10469651e+00
   -2.2150

##### Define trainable as parameters to optimization function